# Tensorflow の script mode での学習と推論を SageMaker で行う

#### ノートブックに含まれる内容

- Tensorflow のscript mode での学習と推論を SageMaker で行うやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: MNIST
- データ: CNN

# セットアップ

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

## 学習データの準備

npy 形式の MNIST データセットが，すでにサンプルデータとして用意されているので，これを使用します．
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [ ]:
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

## 学習用スクリプトの確認

Amazon SageMaker の Script Mode では、トレーニングに用いるコードが実行時に Python スクリプトとして実行されます。その際、データ・モデルの入出力は [こちら](https://sagemaker.readthedocs.io/en/stable/using_tf.html#preparing-a-script-mode-training-script) に記述があるよう SM_CHANNEL_XXXX や SM_MODEL_DIR という環境変数を参照する必要があります。そのため、argparse.ArgumentParser で渡された環境変数と、スクリプト実行時のハイパーパラメータを取得します。

![data-model](./sagemaker-data-model.png)

In [ ]:
!pygmentize 'mnist.py'

## モデルの学習を実行

Tensorflow クラスのオブジェクトを作って，fit() で学習を実施します．

In [ ]:
from sagemaker.tensorflow import TensorFlow


mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m4.xlarge',
                             framework_version='1.13',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': False}},
                             hyperparameters={'n_steps': 10})

In [ ]:
mnist_estimator.fit(training_data_uri)

## モデルの推論を実行

推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．モデルのデプロイには 10 分程度時間がかかります．script mode では，内部的に Tensorflow Serving を用いているため，saved_model() で出力したモデルは，デプロイ時にそのまま Tensorflow Serving 経由でロードされます．詳細については[こちら](https://sagemaker.readthedocs.io/en/stable/using_tf.html#deploying-tensorflow-serving-models)をご覧ください．

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

推論 API のインプット，アウトプットフォーマットは共に [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest) の predict メソッドと同様です．さらに SageMaker では，それ以外の入力フォーマットにも対応しています．例えば json や jsonlines，csv データなどです．このモデルでは numpy 配列の形式で入力を指定しています．より詳細な推論のサンプルは[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint)をご覧ください．

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = predictions['predictions'][i]['classes']
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)